In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

In [4]:
genre_dict = {}
genre_dict['Action-Adventure'] = 0
genre_dict['Romance'] = 1
genre_dict['Horror-Thriller'] = 2
genre_dict['Comedy'] = 3
genre_dict['Science Fiction'] = 4
#genre_dict['Drama'] = 5
genre_dict

{'Action-Adventure': 0,
 'Romance': 1,
 'Horror-Thriller': 2,
 'Comedy': 3,
 'Science Fiction': 4}

In [5]:
genre_map = {}
genre_map['Adventure'] = 'Action-Adventure'
genre_map['Romance'] = 'Romance'
genre_map['Horror'] = 'Horror-Thriller'
genre_map['Thriller'] = 'Horror-Thriller'
genre_map['Comedy'] = 'Comedy'
#genre_map['War'] = 'Action-Adventure'#not sure about this
genre_map['Action'] = 'Action-Adventure'
genre_map['Science Fiction'] = 'Science Fiction'
#genre_map['Drama'] = 'Drama'

In [6]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    return ret

In [7]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [8]:
#put to lower case, remove punctation
def cleanText(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text)
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [9]:
all_data = all_data[all_data.genres_vect.map(sum) > 0]

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 2, size=word_vec_len)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 150

x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [22]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [23]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [24]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

model_cnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
#e = Embedding(num_words_kept, word_vec_len, input_length=max_seq_len, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(len(genre_dict), activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 2s - loss: 2.0752 - raw_multi_label_accuracy: 0.1131 - val_loss: 1.7520 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 2s - loss: 1.5738 - raw_multi_label_accuracy: 0.0873 - val_loss: 1.3526 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 1s - loss: 1.2372 - raw_multi_label_accuracy: 0.0800 - val_loss: 1.0825 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 1s - loss: 1.0080 - raw_multi_label_accuracy: 0.0626 - val_loss: 0.9024 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 1s - loss: 0.8559 - raw_multi_label_accuracy: 0.0500 - val_loss: 0.7847 - val_raw_multi_label_accura

Epoch 55/1000
 - 1s - loss: 0.3791 - raw_multi_label_accuracy: 0.5398 - val_loss: 0.9258 - val_raw_multi_label_accuracy: 0.1735
Epoch 56/1000
 - 1s - loss: 0.3751 - raw_multi_label_accuracy: 0.5533 - val_loss: 0.9231 - val_raw_multi_label_accuracy: 0.1720
Epoch 57/1000
 - 1s - loss: 0.3703 - raw_multi_label_accuracy: 0.5565 - val_loss: 0.9055 - val_raw_multi_label_accuracy: 0.1724
Epoch 58/1000
 - 1s - loss: 0.3712 - raw_multi_label_accuracy: 0.5601 - val_loss: 0.9049 - val_raw_multi_label_accuracy: 0.1814
Epoch 59/1000
 - 1s - loss: 0.3668 - raw_multi_label_accuracy: 0.5695 - val_loss: 0.9118 - val_raw_multi_label_accuracy: 0.1679
Epoch 60/1000
 - 1s - loss: 0.3638 - raw_multi_label_accuracy: 0.5927 - val_loss: 0.9553 - val_raw_multi_label_accuracy: 0.1896
Epoch 61/1000
 - 1s - loss: 0.3579 - raw_multi_label_accuracy: 0.5992 - val_loss: 0.9422 - val_raw_multi_label_accuracy: 0.1627
Epoch 62/1000
 - 1s - loss: 0.3573 - raw_multi_label_accuracy: 0.5983 - val_loss: 0.9226 - val_raw_multi

Epoch 119/1000
 - 1s - loss: 0.2228 - raw_multi_label_accuracy: 0.7810 - val_loss: 1.1959 - val_raw_multi_label_accuracy: 0.2436
Epoch 120/1000
 - 1s - loss: 0.2175 - raw_multi_label_accuracy: 0.7942 - val_loss: 1.2211 - val_raw_multi_label_accuracy: 0.2426
Epoch 121/1000
 - 1s - loss: 0.2200 - raw_multi_label_accuracy: 0.7794 - val_loss: 1.0904 - val_raw_multi_label_accuracy: 0.2492
Epoch 122/1000
 - 1s - loss: 0.2164 - raw_multi_label_accuracy: 0.7905 - val_loss: 1.2383 - val_raw_multi_label_accuracy: 0.2397
Epoch 123/1000
 - 1s - loss: 0.2161 - raw_multi_label_accuracy: 0.7831 - val_loss: 1.2586 - val_raw_multi_label_accuracy: 0.2502
Epoch 124/1000
 - 1s - loss: 0.2134 - raw_multi_label_accuracy: 0.7843 - val_loss: 1.1895 - val_raw_multi_label_accuracy: 0.2438
Epoch 125/1000
 - 1s - loss: 0.2145 - raw_multi_label_accuracy: 0.7868 - val_loss: 1.1768 - val_raw_multi_label_accuracy: 0.2432
Epoch 126/1000
 - 1s - loss: 0.2136 - raw_multi_label_accuracy: 0.7947 - val_loss: 1.2283 - val_r

Epoch 183/1000
 - 1s - loss: 0.1666 - raw_multi_label_accuracy: 0.8393 - val_loss: 1.5177 - val_raw_multi_label_accuracy: 0.2627
Epoch 184/1000
 - 1s - loss: 0.1624 - raw_multi_label_accuracy: 0.8407 - val_loss: 1.4433 - val_raw_multi_label_accuracy: 0.2613
Epoch 185/1000
 - 1s - loss: 0.1626 - raw_multi_label_accuracy: 0.8424 - val_loss: 1.4950 - val_raw_multi_label_accuracy: 0.2701
Epoch 186/1000
 - 1s - loss: 0.1595 - raw_multi_label_accuracy: 0.8442 - val_loss: 1.4621 - val_raw_multi_label_accuracy: 0.2696
Epoch 187/1000
 - 1s - loss: 0.1628 - raw_multi_label_accuracy: 0.8436 - val_loss: 1.4097 - val_raw_multi_label_accuracy: 0.2581
Epoch 188/1000
 - 1s - loss: 0.1616 - raw_multi_label_accuracy: 0.8435 - val_loss: 1.4413 - val_raw_multi_label_accuracy: 0.2646
Epoch 189/1000
 - 1s - loss: 0.1610 - raw_multi_label_accuracy: 0.8449 - val_loss: 1.5293 - val_raw_multi_label_accuracy: 0.2759
Epoch 190/1000
 - 1s - loss: 0.1616 - raw_multi_label_accuracy: 0.8459 - val_loss: 1.5658 - val_r

In [25]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [26]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [27]:
y_test[:,0].sum()

187

In [28]:
predictions[:,0].sum()

256

In [29]:
predictions[0]

array([0, 1, 1, 0, 0])

In [30]:
multi_label_accuracy(y_test, predictions)

0.3321256038647344

In [31]:
multi_label_precision(y_test, predictions)

0.42822636300897177

In [32]:
multi_label_recall(y_test, predictions)

0.4366804692891649

In [33]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 64.59627329192547


In [34]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.5755693581780539
Precision for Action-Adventure: 0.46484375
Recall for Action-Adventure: 0.6363636363636364

Accuruacy for Romance: 0.722567287784679
Precision for Romance: 0.3157894736842105
Recall for Romance: 0.22641509433962265

Accuruacy for Horror-Thriller: 0.505175983436853
Precision for Horror-Thriller: 0.41637010676156583
Recall for Horror-Thriller: 0.609375

Accuruacy for Comedy: 0.5631469979296067
Precision for Comedy: 0.5083333333333333
Recall for Comedy: 0.2863849765258216

Accuruacy for Science Fiction: 0.8633540372670807
Precision for Science Fiction: 0.25
Recall for Science Fiction: 0.08928571428571429



CNN but with multiple filter sizes so we don't just filter on group of words at a time

In [35]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

model_input = Input(shape=(max_seq_len,), dtype='int32')

e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)(model_input)
two_word_filter = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(e)
two_word_filter = GlobalMaxPooling1D()(two_word_filter)
three_word_filter = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(e)
three_word_filter = GlobalMaxPooling1D()(three_word_filter)
four_word_filter = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(e)
four_word_filter = GlobalMaxPooling1D()(four_word_filter)
merged = concatenate([two_word_filter, three_word_filter, four_word_filter], axis=1)

merged = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dropout(0.5)(merged)
merged = Dense(len(genre_dict))(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[model_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])

In [36]:
model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 4s - loss: 2.8990 - raw_multi_label_accuracy: 0.0838 - val_loss: 2.2934 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 4s - loss: 1.9273 - raw_multi_label_accuracy: 0.0526 - val_loss: 1.5416 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 4s - loss: 1.3211 - raw_multi_label_accuracy: 0.0315 - val_loss: 1.0943 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 4s - loss: 0.9764 - raw_multi_label_accuracy: 0.0274 - val_loss: 0.8450 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 4s - loss: 0.7841 - raw_multi_label_accuracy: 0.0235 - val_loss: 0.7215 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 4s - loss: 0.6850 - raw_multi_label_accuracy: 0.0255 - val_loss: 0.6478 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 4s - loss: 0.6351 - raw_multi_label_accuracy: 0.0510 - val_loss: 0.6192 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 8/1000
 - 4s - loss: 0.5987 - 

Epoch 65/1000
 - 4s - loss: 0.0366 - raw_multi_label_accuracy: 0.9825 - val_loss: 0.9537 - val_raw_multi_label_accuracy: 0.3329
Epoch 66/1000
 - 4s - loss: 0.0355 - raw_multi_label_accuracy: 0.9815 - val_loss: 0.9386 - val_raw_multi_label_accuracy: 0.3520
Epoch 67/1000
 - 4s - loss: 0.0320 - raw_multi_label_accuracy: 0.9857 - val_loss: 0.9994 - val_raw_multi_label_accuracy: 0.3306
Epoch 68/1000
 - 4s - loss: 0.0357 - raw_multi_label_accuracy: 0.9792 - val_loss: 0.9313 - val_raw_multi_label_accuracy: 0.3296
Epoch 69/1000
 - 4s - loss: 0.0326 - raw_multi_label_accuracy: 0.9867 - val_loss: 0.9982 - val_raw_multi_label_accuracy: 0.3266
Epoch 70/1000
 - 4s - loss: 0.0321 - raw_multi_label_accuracy: 0.9840 - val_loss: 0.9896 - val_raw_multi_label_accuracy: 0.3387
Epoch 71/1000
 - 4s - loss: 0.0310 - raw_multi_label_accuracy: 0.9861 - val_loss: 0.9713 - val_raw_multi_label_accuracy: 0.3315
Epoch 72/1000
 - 4s - loss: 0.0305 - raw_multi_label_accuracy: 0.9874 - val_loss: 1.0366 - val_raw_multi

Epoch 129/1000
 - 4s - loss: 0.0242 - raw_multi_label_accuracy: 0.9885 - val_loss: 1.2798 - val_raw_multi_label_accuracy: 0.3217
Epoch 130/1000
 - 4s - loss: 0.0218 - raw_multi_label_accuracy: 0.9934 - val_loss: 1.3802 - val_raw_multi_label_accuracy: 0.2890
Epoch 131/1000
 - 4s - loss: 0.0222 - raw_multi_label_accuracy: 0.9892 - val_loss: 1.2538 - val_raw_multi_label_accuracy: 0.3293
Epoch 132/1000
 - 4s - loss: 0.0288 - raw_multi_label_accuracy: 0.9840 - val_loss: 1.3560 - val_raw_multi_label_accuracy: 0.3060
Epoch 133/1000
 - 4s - loss: 0.0266 - raw_multi_label_accuracy: 0.9910 - val_loss: 1.3333 - val_raw_multi_label_accuracy: 0.3256
Epoch 134/1000
 - 4s - loss: 0.0238 - raw_multi_label_accuracy: 0.9930 - val_loss: 1.3119 - val_raw_multi_label_accuracy: 0.3492
Epoch 135/1000
 - 4s - loss: 0.0215 - raw_multi_label_accuracy: 0.9927 - val_loss: 1.2455 - val_raw_multi_label_accuracy: 0.3542
Epoch 136/1000
 - 4s - loss: 0.0210 - raw_multi_label_accuracy: 0.9922 - val_loss: 1.3654 - val_r

Epoch 193/1000
 - 4s - loss: 0.0211 - raw_multi_label_accuracy: 0.9918 - val_loss: 1.6954 - val_raw_multi_label_accuracy: 0.3380
Epoch 194/1000
 - 4s - loss: 0.0256 - raw_multi_label_accuracy: 0.9891 - val_loss: 1.6477 - val_raw_multi_label_accuracy: 0.3172
Epoch 195/1000
 - 4s - loss: 0.0249 - raw_multi_label_accuracy: 0.9919 - val_loss: 1.5984 - val_raw_multi_label_accuracy: 0.3140
Epoch 196/1000
 - 4s - loss: 0.0242 - raw_multi_label_accuracy: 0.9929 - val_loss: 1.6638 - val_raw_multi_label_accuracy: 0.3121
Epoch 197/1000
 - 4s - loss: 0.0241 - raw_multi_label_accuracy: 0.9912 - val_loss: 1.6545 - val_raw_multi_label_accuracy: 0.2815
Epoch 198/1000
 - 4s - loss: 0.0221 - raw_multi_label_accuracy: 0.9937 - val_loss: 1.6714 - val_raw_multi_label_accuracy: 0.3044
Epoch 199/1000
 - 4s - loss: 0.0183 - raw_multi_label_accuracy: 0.9949 - val_loss: 1.6279 - val_raw_multi_label_accuracy: 0.3238
Epoch 200/1000
 - 4s - loss: 0.0197 - raw_multi_label_accuracy: 0.9927 - val_loss: 1.5487 - val_r

In [37]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [38]:
multi_label_accuracy(y_test, predictions)

0.3321256038647344

In [39]:
multi_label_precision(y_test, predictions)

0.42822636300897177

In [40]:
multi_label_recall(y_test, predictions)

0.4366804692891649

In [41]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 64.59627329192547


In [42]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.5755693581780539
Precision for Action-Adventure: 0.46484375
Recall for Action-Adventure: 0.6363636363636364

Accuruacy for Romance: 0.722567287784679
Precision for Romance: 0.3157894736842105
Recall for Romance: 0.22641509433962265

Accuruacy for Horror-Thriller: 0.505175983436853
Precision for Horror-Thriller: 0.41637010676156583
Recall for Horror-Thriller: 0.609375

Accuruacy for Comedy: 0.5631469979296067
Precision for Comedy: 0.5083333333333333
Recall for Comedy: 0.2863849765258216

Accuruacy for Science Fiction: 0.8633540372670807
Precision for Science Fiction: 0.25
Recall for Science Fiction: 0.08928571428571429

